# Visualize the molecule

In [40]:
from rdkit import Chem
import py3Dmol
mol_folder = './Molecules/'
mol_fname = '50A12Rot_3.mol2'
mol_filepath = mol_folder + mol_fname
mol = Chem.MolFromMol2File(mol_filepath)
mblock = Chem.MolToMolBlock(mol)
view = py3Dmol.view(width=500, height=250)
view.addModel(mblock, 'mol')
view.setStyle({'stick':{}})
view.zoomTo()
view.show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Input Variables

In [41]:
n_atoms_mol = mol.GetNumAtoms()  # no. of atoms in the molecule (excluding hydrogen atoms)
print('No. of atoms: ', n_atoms_mol)
conformers = mol.GetConformers()
conf = conformers[0]
coords_mol = {}  # Coordinates of the atoms
for i in range(n_atoms_mol):
    coords_mol[i] = list(conf.GetAtomPosition(i))
print('Coordinates of atoms:')
for i in coords_mol:
    print(i, '\t:', coords_mol[i])

No. of atoms:  50
Coordinates of atoms:
0 	: [4.669, -2.683, -1.244]
1 	: [4.802, -1.221, -1.001]
2 	: [5.88, -0.421, -1.016]
3 	: [7.351, -0.665, -1.165]
4 	: [7.848, -2.11, -1.154]
5 	: [9.277, -2.08, -1.082]
6 	: [9.917, -3.277, -0.933]
7 	: [9.334, -4.538, -1.025]
8 	: [10.112, -5.69, -0.843]
9 	: [11.483, -5.601, -0.566]
10 	: [12.061, -4.328, -0.487]
11 	: [11.284, -3.18, -0.672]
12 	: [12.264, -6.767, -0.359]
13 	: [12.06, -7.956, -1.019]
14 	: [11.27, -8.201, -1.92]
15 	: [12.973, -9.002, -0.449]
16 	: [13.91, -8.218, 0.447]
17 	: [14.218, -9.006, 2.067]
18 	: [12.596, -8.767, 2.883]
19 	: [12.583, -9.406, 4.267]
20 	: [11.692, -10.191, 4.579]
21 	: [13.606, -8.953, 5.083]
22 	: [13.847, -9.297, 6.427]
23 	: [12.931, -9.979, 7.229]
24 	: [13.231, -10.267, 8.563]
25 	: [14.45, -9.872, 9.109]
26 	: [15.386, -9.187, 8.323]
27 	: [15.068, -8.897, 6.984]
28 	: [16.695, -8.729, 8.914]
29 	: [17.136, -9.525, 9.928]
30 	: [16.617, -7.472, 9.431]
31 	: [17.708, -8.695, 8.003]
32 	: [13.

In [42]:
input_folder = './Input_Rot_2/'
input_fname = mol_fname[:-5] + '_input.txt'
input_filepath = input_folder + input_fname
input_lines = open(input_filepath, 'r').readlines()
torsional_bonds_mol = eval(input_lines[0])
median_list_mol = []
for atoms in input_lines[1].split(','):
    median_list_mol.append(int(atoms))
coords_rotation_mol = eval(input_lines[2])
distance_pairs_mol = eval(input_lines[3])
print(torsional_bonds_mol)
print(median_list_mol)
print(coords_rotation_mol)
print(distance_pairs_mol)

{0: (21, 22), 1: (37, 38)}
[0, 11, 14, 20, 21, 22, 30, 33, 35, 37, 38, 46]
{23: [0], 24: [0], 25: [0], 26: [0], 27: [0], 28: [0], 29: [0], 30: [0], 31: [0], 39: [1], 40: [1], 41: [1], 42: [1], 43: [1], 44: [1], 45: [1], 46: [1], 47: [1], 48: [1], 49: [1]}
{0: [30, 46], 11: [30, 46], 14: [30, 46], 20: [30, 46], 21: [30, 46], 22: [46], 30: [33, 35, 37, 38, 46], 33: [46], 35: [46], 37: [46], 38: [], 46: []}


In [43]:
# torsional_bonds_mol = { 0: (9, 10), 1: (19, 20)  }  # Torsional bond numbers and their respective coordinate ends
# median_list_mol = [2, 6, 9, 10, 14, 19, 20, 23]  # List of atoms (median) considered for the HUBO construction
# coords_rotation_mol = {11:[0], 12:[0], 13:[0], 14:[0], 15:[0], 16:[0], 17:[0], 18:[0], 19:[0], 20:[0], 21:[0,1], 22:[0,1], 23:[0,1], 24:[0,1] }  # atom index and the rotatable bond numbers list that affect them
# distance_pairs_mol = {2:[14, 19, 20, 23], 6:[14, 19, 20, 23], 9:[14, 19, 20, 23], 10:[23], 14:[23], 19:[23], 20:[], 23:[] }  # indices dictionary to find distances

In [44]:
import copy

coords_median = {}
for i, index in enumerate(median_list_mol):
    coords_median[i] = coords_mol[index]
coords_median_2 = copy.deepcopy(coords_median)
print('coords_median: ', coords_median)

torsional_bonds = {}
for i in torsional_bonds_mol:
    torsional_bonds[i] = (median_list_mol.index(torsional_bonds_mol[i][0]), median_list_mol.index(torsional_bonds_mol[i][1]))
print('\ntorsional_bonds: ', torsional_bonds)

coords_rotation_dict = {}
for atom in median_list_mol:
    if atom in coords_rotation_mol.keys():
        coords_rotation_dict[median_list_mol.index(atom)] = coords_rotation_mol[atom]
print('\ncoords_rotation_dict: ', coords_rotation_dict)

distance_pairs = []
for x, y in distance_pairs_mol.items():
    u = median_list_mol.index(x)
    for i in y:
        v = median_list_mol.index(i)
        distance_pairs.append((u, v))
print('\ndistance pairs: ', distance_pairs)



coords_median:  {0: [4.669, -2.683, -1.244], 1: [11.284, -3.18, -0.672], 2: [11.27, -8.201, -1.92], 3: [11.692, -10.191, 4.579], 4: [13.606, -8.953, 5.083], 5: [13.847, -9.297, 6.427], 6: [16.617, -7.472, 9.431], 7: [13.654, -5.995, 1.357], 8: [6.192, 1.921, -0.549], 9: [3.628, -0.391, -0.802], 10: [2.617, -0.872, 0.527], 11: [-2.804, 2.045, -0.803]}

torsional_bonds:  {0: (4, 5), 1: (9, 10)}

coords_rotation_dict:  {6: [0], 11: [1]}

distance pairs:  [(0, 6), (0, 11), (1, 6), (1, 11), (2, 6), (2, 11), (3, 6), (3, 11), (4, 6), (4, 11), (5, 11), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (7, 11), (8, 11), (9, 11)]


In [45]:
import sympy as sp
n_bonds = len(torsional_bonds)  # no. of bonds
n_angles = 8  # no. of discrete angles
x = sp.symbols(f'x(0:{n_bonds*n_angles})')  # hubo variables
print(x)

(x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15)


# Function to generate hard constraint

In [46]:
# generate hard constraint
def generate_hard_hubo():
    hard_constraint = 0
    index = 0
    for i in range(n_bonds):
        summation = 0
        for j in range(n_angles):
            summation += x[index]
            index += 1
        hard_constraint += (summation - 1) ** 2
    a_const = sp.Symbol('A_const')
    hard_constraint *= a_const
    return hard_constraint

# Functions to rotate coordinates with hubo variables

In [47]:
def rotate_coordinates(rotation_matrix, old_coords):
    coord_vector = ones(4, 1)
    coord_vector[0, 0] = old_coords[0]
    coord_vector[1, 0] = old_coords[1]
    coord_vector[2, 0] = old_coords[2]
    coord_rot_vector = sp.expand(rotation_matrix * coord_vector)
    return [coord_rot_vector[0, 0], coord_rot_vector[1, 0], coord_rot_vector[2, 0]]

In [48]:
def generate_thetas():
    angle_incr = 2 * sp.pi / n_angles
    thetas = [i*angle_incr for i in range(n_angles)]
    return thetas

In [49]:
def rotation_matrix_hubo(first_coords, second_coords, bond_no):
    x_dash, y_dash, z_dash = first_coords[0], first_coords[1], first_coords[2]
    x_ddash, y_ddash, z_ddash = second_coords[0], second_coords[1], second_coords[2]
    dx = x_ddash - x_dash
    dy = y_ddash - y_dash
    dz = z_ddash - z_dash
    l_sq = dx ** 2 + dy ** 2 + dz ** 2
    l = sp.sqrt(l_sq)
    thetas = generate_thetas()
    c_theta = 0.0
    s_theta = 0.0
    index = n_angles * bond_no
    for i in range(n_angles):
        c_theta += sp.cos(thetas[i]) * x[index]
        s_theta += sp.sin(thetas[i]) * x[index]
        index += 1
    rotation_matrix = eye(4)
    rotation_matrix[0, 0] = (dx ** 2 + (dy ** 2 + dz ** 2) * c_theta) / l_sq
    rotation_matrix[0, 1] = (dx * dy * (1 - c_theta) - dz * l * s_theta) / l_sq
    rotation_matrix[0, 2] = (dx * dz * (1 - c_theta) + dy * l * s_theta) / l_sq
    rotation_matrix[0, 3] = ((x_dash * (dy ** 2 + dz ** 2) - dx * (y_dash * dy + z_dash * dz)) * (1 - c_theta) + (
            y_dash * dz - z_dash * dy) * l * s_theta) / l_sq
    rotation_matrix[1, 0] = (dx * dy * (1 - c_theta) + dz * l * s_theta) / l_sq
    rotation_matrix[1, 1] = (dy ** 2 + (dx ** 2 + dz ** 2) * c_theta) / l_sq
    rotation_matrix[1, 2] = (dy * dz * (1 - c_theta) - dx * l * s_theta) / l_sq
    rotation_matrix[1, 3] = ((y_dash * (dx ** 2 + dz ** 2) - dy * (x_dash * dx + z_dash * dz)) * (1 - c_theta) + (
            z_dash * dx - x_dash * dz) * l * s_theta) / l_sq
    rotation_matrix[2, 0] = (dx * dz * (1 - c_theta) - dy * l * s_theta) / l_sq
    rotation_matrix[2, 1] = (dy * dz * (1 - c_theta) + dx * l * s_theta) / l_sq
    rotation_matrix[2, 2] = (dz ** 2 + (dx ** 2 + dy ** 2) * c_theta) / l_sq
    rotation_matrix[2, 3] = ((z_dash * (dx ** 2 + dy ** 2) - dz * (x_dash * dx + y_dash * dy)) * (1 - c_theta) + (
            x_dash * dy - y_dash * dx) * l * s_theta) / l_sq
    return rotation_matrix

In [50]:
from sympy.matrices import ones, eye
import time
start = time.time()

for i in coords_rotation_dict:
    rot_mat = eye(4, 4)
    for bond_no in coords_rotation_dict[i]:
        temp_rot_mat = rotation_matrix_hubo(coords_median[torsional_bonds[bond_no][0]], coords_median[torsional_bonds[bond_no][1]], bond_no)
        rot_mat = rot_mat * temp_rot_mat
    coords_median_2[i] = rotate_coordinates(rot_mat, coords_median[i])

end = time.time()
print("The time of execution: ", (end-start), "seconds")

The time of execution:  1.0741002559661865 seconds


# Functions to generate the optimization contraint in hubo

In [51]:
def distance_squared(first_coords, second_coords):
    dis_sq = (second_coords[0] - first_coords[0])**2 + (second_coords[1] - first_coords[1])**2 + (second_coords[2] - first_coords[2])**2
    return dis_sq

def generate_distance_hubo():
    distance_sq = 0
    for pair in distance_pairs:
        distance_sq += distance_squared(coords_median_2[pair[0]], coords_median_2[pair[1]])
    return distance_sq.expand()

In [52]:
start = time.time()

hard_constraint = generate_hard_hubo()
distance_constraint = generate_distance_hubo()
hubo_expr = sp.expand(hard_constraint - distance_constraint)

end = time.time()
print("The time of execution: ", (end-start), "seconds")

The time of execution:  3.8760340213775635 seconds


# The full hubo expression is written in the file 'full_hubo_expr.txt'

In [53]:
def hubo_expr_to_dict():
    hubo_args = hubo_expr.args
    hubo_dict = {}
    for monom in hubo_args:
        dict_value = monom.as_coeff_mul()[0]
        monom_key = []
        monom_coeffs = monom.as_coeff_mul()[1]
        for monom_item in monom_coeffs:
            if re.match("^x(\d*)\d$", str(monom_item)):
                monom_key.append(int(str(monom_item)[1:]))
            elif re.match("^x(\d*)\\*\\*(\d)$", str(monom_item)):
                monom_key.append(int(str(monom_item).split('**')[0][1:]))
            else:
                dict_value *= monom_item
        if len(monom_key) > 0:
            monom_key.sort()
            dict_key = tuple(monom_key)
        else:
            dict_key = ()
        dict_value = dict_value.evalf()
        if dict_key in hubo_dict:
            hubo_dict[dict_key] += dict_value
        else:
            hubo_dict[dict_key] = dict_value
    return hubo_dict

In [54]:
import re
hubo_dict = hubo_expr_to_dict()

In [55]:
hubo_folder = './Hubo_Rot_2/'
hubo_fname = mol_fname[:-5] + '_hubo.txt'
hubo_filepath = hubo_folder + hubo_fname
f = open(hubo_filepath, "w")
print('full hubo dictionary written in file - ' + hubo_filepath)
f.write(str(hubo_dict))
f.close()

full hubo dictionary written in file - ./Hubo_Rot_2/50A12Rot_3_hubo.txt


In [56]:
#This is used to check the maximum coefficient appearing in Hubo_B
A_const=0
read_dictionary_B = open(hubo_filepath, 'r').read()
HUBO_B=eval(read_dictionary_B)

#We set the Hard constraint strength as the (maximum coefficient appearing in Hubo_B)*const.
#const was empirically selected to be 10
const=10
A_const=max(map(abs, list(HUBO_B.values())))*const
# A_const = 1000

#read the final HUBO
read_dictionary= open(hubo_filepath, 'r').read()
HUBO=eval(read_dictionary)

In [57]:
print("Current size of the HUBO:",len(HUBO)) 

Current size of the HUBO: 137


In [58]:
def threshold_approx(h, val=1):
    d =h.copy()
    monoms = h.keys()
    for m in monoms:     
        temp = d[m]
        if (temp < 0.0):
            temp = -1.0 * temp
        if (temp <= (10.0 ** (val))):
            del d[m]
    return d

In [59]:
#Coefficints with absolute value less than 10^{threshold} are deleted from the HUBO.
threshold=2

HUBO=threshold_approx(HUBO,threshold)

print("Size of the HUBO after threshold approximation:",len(HUBO))

Size of the HUBO after threshold approximation: 73


In [60]:
import dimod
#calculate the strength parameter needed by make_quadratic
max_hubo_value=max(map(abs, list(HUBO.values())))
strength=1.5*max_hubo_value
#generate the bqm
bqm = dimod.make_quadratic(HUBO, strength, dimod.BINARY)

In [61]:
def find_bond_theta_soln(solution):
    thetas = generate_thetas()
    bond_theta_soln = {}
    for key, value in solution.items():
        if solution[key] == 1:
            bond_theta_soln[key//n_angles] = thetas[key%n_angles]
    return bond_theta_soln

In [62]:
import neal
sampler = neal.SimulatedAnnealingSampler()
sample_size=10
start = time.time()
sampleset = sampler.sample(bqm, num_reads=sample_size)
end = time.time()
print("The time of execution: ", (end-start), "s")
print(sampleset)

The time of execution:  0.036550045013427734 s
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15       energy num_oc.
2  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1 -4288.983641       1
6  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1 -4288.983641       1
8  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  1 -4041.426123       1
4  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0 -3784.549764       1
7  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0  0 -3682.008083       1
0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0 -3565.227373       1
3  0  0  0  0  1  0  0  0  0  0  1  0  0  0  0  0 -3360.621672       1
9  0  1  0  0  0  0  0  0  0  0  1  0  0  0  0  0 -3113.064154       1
1  1  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0 -2750.962554       1
5  0  0  0  1  0  0  0  0  0  0  0  0  0  1  0  0 -2429.576143       1
['BINARY', 10 rows, 10 samples, 16 variables]


In [63]:
sa_soln = sampleset.first.sample
print(sa_soln)
bond_theta_soln = find_bond_theta_soln(sa_soln)
print(bond_theta_soln)

{0: 0, 1: 0, 2: 0, 3: 0, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 1}
{0: pi, 1: 7*pi/4}


In [64]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler = EmbeddingComposite(DWaveSampler())
start = time.time()
sampleset = sampler.sample(bqm, num_reads=1000)
end = time.time()
print("The time of execution: ", (end-start), "s")
print(sampleset.slice(10))

The time of execution:  0.19040489196777344 s
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15       energy num_oc. ...
0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  0  0 -4983.354702       6 ...
1  0  0  0  0  0  0  0  1  1  0  0  0  0  0  0  0  -4880.81302       3 ...
2  0  0  0  0  0  1  0  0  1  0  0  0  0  0  0  0 -4850.232092       6 ...
3  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  1 -4712.911733      15 ...
4  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  1 -4610.370052       9 ...
5  1  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0 -4602.674574      12 ...
6  0  0  0  0  0  0  1  0  0  1  0  0  0  0  0  0 -4598.814583       7 ...
7  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  1 -4579.789123       8 ...
8  0  0  0  0  1  0  0  0  1  0  0  0  0  0  0  0 -4559.426609       2 ...
9  0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  0 -4496.272902       9 ...
['BINARY', 10 rows, 77 samples, 16 variables]


In [65]:
qa_soln = sampleset.first.sample
print(qa_soln)
bond_theta_soln = find_bond_theta_soln(qa_soln)
print(bond_theta_soln)

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 0, 8: 1, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0}
{0: 3*pi/2, 1: 0}


In [66]:
def rotation_matrix_new_coords(first_coords, second_coords, bond_no, soln_theta):
    x_dash, y_dash, z_dash = first_coords[0], first_coords[1], first_coords[2]
    x_ddash, y_ddash, z_ddash = second_coords[0], second_coords[1], second_coords[2]
    dx = x_ddash - x_dash
    dy = y_ddash - y_dash
    dz = z_ddash - z_dash
    l_sq = dx ** 2 + dy ** 2 + dz ** 2
    l = sp.sqrt(l_sq)
    c_theta = sp.cos(soln_theta)
    s_theta = sp.sin(soln_theta)
    index = n_angles * bond_no
    rotation_matrix = eye(4)
    rotation_matrix[0, 0] = ((dx ** 2 + (dy ** 2 + dz ** 2) * c_theta) / l_sq).evalf()
    rotation_matrix[0, 1] = ((dx * dy * (1 - c_theta) - dz * l * s_theta) / l_sq).evalf()
    rotation_matrix[0, 2] = ((dx * dz * (1 - c_theta) + dy * l * s_theta) / l_sq).evalf()
    rotation_matrix[0, 3] = (((x_dash * (dy ** 2 + dz ** 2) - dx * (y_dash * dy + z_dash * dz)) * (1 - c_theta) + (
            y_dash * dz - z_dash * dy) * l * s_theta) / l_sq).evalf()
    rotation_matrix[1, 0] = ((dx * dy * (1 - c_theta) + dz * l * s_theta) / l_sq).evalf()
    rotation_matrix[1, 1] = ((dy ** 2 + (dx ** 2 + dz ** 2) * c_theta) / l_sq).evalf()
    rotation_matrix[1, 2] = ((dy * dz * (1 - c_theta) - dx * l * s_theta) / l_sq).evalf()
    rotation_matrix[1, 3] = (((y_dash * (dx ** 2 + dz ** 2) - dy * (x_dash * dx + z_dash * dz)) * (1 - c_theta) + (
            z_dash * dx - x_dash * dz) * l * s_theta) / l_sq).evalf()
    rotation_matrix[2, 0] = ((dx * dz * (1 - c_theta) - dy * l * s_theta) / l_sq).evalf()
    rotation_matrix[2, 1] = ((dy * dz * (1 - c_theta) + dx * l * s_theta) / l_sq).evalf()
    rotation_matrix[2, 2] = ((dz ** 2 + (dx ** 2 + dy ** 2) * c_theta) / l_sq).evalf()
    rotation_matrix[2, 3] = (((z_dash * (dx ** 2 + dy ** 2) - dz * (x_dash * dx + y_dash * dy)) * (1 - c_theta) + (
            x_dash * dy - y_dash * dx) * l * s_theta) / l_sq).evalf()
    return rotation_matrix

In [67]:
def compute_new_coords(solution):
    final_coords = copy.deepcopy(coords_mol)
    for i in coords_rotation_mol:
        if len(coords_rotation_mol[i]) > 0:
            rot_mat = eye(4, 4)
            for bond_no in coords_rotation_mol[i]:
                temp_rot_mat = rotation_matrix_new_coords(coords_mol[torsional_bonds_mol[bond_no][0]], coords_mol[torsional_bonds_mol[bond_no][1]], bond_no, solution[bond_no])
                rot_mat = rot_mat * temp_rot_mat
            final_coords[i] = rotate_coordinates(rot_mat, coords_mol[i])
    return final_coords


In [68]:
compute_new_coords(bond_theta_soln)

{0: [4.669, -2.683, -1.244],
 1: [4.802, -1.221, -1.001],
 2: [5.88, -0.421, -1.016],
 3: [7.351, -0.665, -1.165],
 4: [7.848, -2.11, -1.154],
 5: [9.277, -2.08, -1.082],
 6: [9.917, -3.277, -0.933],
 7: [9.334, -4.538, -1.025],
 8: [10.112, -5.69, -0.843],
 9: [11.483, -5.601, -0.566],
 10: [12.061, -4.328, -0.487],
 11: [11.284, -3.18, -0.672],
 12: [12.264, -6.767, -0.359],
 13: [12.06, -7.956, -1.019],
 14: [11.27, -8.201, -1.92],
 15: [12.973, -9.002, -0.449],
 16: [13.91, -8.218, 0.447],
 17: [14.218, -9.006, 2.067],
 18: [12.596, -8.767, 2.883],
 19: [12.583, -9.406, 4.267],
 20: [11.692, -10.191, 4.579],
 21: [13.606, -8.953, 5.083],
 22: [13.847, -9.297, 6.427],
 23: [13.5246753466952, -8.47484862545528, 7.50753594813231],
 24: [13.8144361746317, -8.87366714949978, 8.81500772506388],
 25: [14.4332294541536, -10.0984346020590, 9.05405074288743],
 26: [14.7654005374591, -10.9413251220330, 7.98525939620016],
 27: [14.4748994546045, -10.5241439078197, 6.67388074936782],
 28: [15.4

In [69]:
def find_volume_change(old_coords, new_coords):
    n = len(old_coords)
    old_distance_sq = 0
    for i in range(n-1):
        for j in range(i+1, n):
            old_distance_sq += distance_squared(old_coords[i], old_coords[j])
    
    new_distance_sq = 0
    for i in range(n-1):
        for j in range(i+1, n):
            new_distance_sq += distance_squared(new_coords[i], new_coords[j])
    # print("new volume: ", new_distance_sq.evalf())
    # print("old volume: ", old_distance_sq)
    # print('change in volume: ', new_distance_sq.evalf() - old_distance_sq)
    return new_distance_sq.evalf() - old_distance_sq

In [70]:
# bond_theta_soln = {0: 0, 1: sp.pi}
# bond_theta_soln = {0: sp.pi/4}
# bond_theta_soln = {0: 2*sp.pi/4}
# bond_theta_soln = {0: 3*sp.pi/4}
# bond_theta_soln = {0: 4*sp.pi/4}
# bond_theta_soln = {0: 5*sp.pi/4}
# bond_theta_soln = {0: 6*sp.pi/4}
# bond_theta_soln = {0: 7*sp.pi/4}

In [71]:
new_coords = compute_new_coords(bond_theta_soln)
volume_change = find_volume_change(coords_mol, new_coords)
print('change in volume: ', volume_change)

change in volume:  3422.30331882919


In [72]:
# brute force solution
start = time.time()
thetas = generate_thetas()
best_volume_change = 0
best_soln = {}
for i in range(8):
    for j in range(8):
        bond_theta_soln = {0: thetas[i], 1: thetas[j]}
        new_coords = compute_new_coords(bond_theta_soln)
        vol_difference = find_volume_change(coords_mol, new_coords)
        print(bond_theta_soln, ': ', vol_difference)
        if vol_difference > best_volume_change:
            best_volume_change = vol_difference
            best_soln = bond_theta_soln
end = time.time()
print("The time of execution: ", (end-start), "s")    
print('best solution: ', best_soln)
print('best volume change: ', best_volume_change)




{0: 0, 1: 0} :  0
{0: 0, 1: pi/4} :  -7400.98284565745
{0: 0, 1: pi/2} :  -23673.5340617109
{0: 0, 1: 3*pi/4} :  -39285.4138402079
{0: 0, 1: pi} :  -45091.3947410423
{0: 0, 1: 5*pi/4} :  -37690.4118953849
{0: 0, 1: 3*pi/2} :  -21417.8606793309
{0: 0, 1: 7*pi/4} :  -5805.98090083429
{0: pi/4, 1: 0} :  -5357.81900856335
{0: pi/4, 1: pi/4} :  -12459.4568369353
{0: pi/4, 1: pi/2} :  -28345.7495871320
{0: pi/4, 1: 3*pi/4} :  -43710.7224219177
{0: pi/4, 1: pi} :  -49553.7826401694
{0: pi/4, 1: 5*pi/4} :  -42452.1448117974
{0: pi/4, 1: 3*pi/2} :  -26565.8520616007
{0: pi/4, 1: 7*pi/4} :  -11200.8792268148
{0: pi/2, 1: 0} :  -9298.07356791379
{0: pi/2, 1: pi/4} :  -16418.1646314156
{0: pi/2, 1: pi/2} :  -32098.6715326786
{0: pi/2, 1: 3*pi/4} :  -47154.1659938273
{0: pi/2, 1: pi} :  -52765.3435477543
{0: pi/2, 1: 5*pi/4} :  -45645.2524842522
{0: pi/2, 1: 3*pi/2} :  -29964.7455829894
{0: pi/2, 1: 7*pi/4} :  -14909.2511218404
{0: 3*pi/4, 1: 0} :  -9512.61599638555
{0: 3*pi/4, 1: pi/4} :  -16958.1

In [73]:
from rdkit.Geometry import Point3D
conf = mol.GetConformer()
for i in range(len(coords_mol)):
    x, y, z = new_coords[i]
    conf.SetAtomPosition(i, Point3D(float(x), float(y), float(z)))

In [74]:
mblock = Chem.MolToMolBlock(mol)
view = py3Dmol.view(width=500, height=250)
view.addModel(mblock, 'mol')
view.setStyle({'stick':{}})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [76]:
mol_filepath = mol_folder + mol_fname
mol = Chem.MolFromMol2File(mol_filepath)
mblock = Chem.MolToMolBlock(mol)
view = py3Dmol.view(width=500, height=250)
view.addModel(mblock, 'mol')
view.setStyle({'stick':{}})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol